# Content-based Filtering Model Final

In [ ]:
import csv
import random
import tensorflow as tf

Create a list of brand choice for one hot encoding.

In [ ]:
user_dict = {}
brand_choice_list = ['Oppo', 'Asus', 'Infinix', 'Samsung', 'Vivo', 'Huawei', 'Apple', 'Realme', 'Xiaomi', 'Poco', 'lainnya/tidak ada']

Create user_list

In [ ]:
user_row = 1507
user_column = 7
user_list = [[0 for _ in range(user_column)] for _ in range(user_row)]

Access user dataset from csv file.

In [ ]:
with open('user_surveys.csv', 'r') as file:
  user_dict['user'] = []
  user_dict['user_fav'] = []
  user_index = []
  csvFile = csv.reader(file)
  next(csvFile)
  for line in csvFile:
    user_index.append(int(line[0])-1)
    user_dict[f'user'].append([int(line[1]), int(line[2]), int(line[3]), int(line[4]), int(line[5]), int(line[6])])
    user_dict[f'user_fav'].append(int(line[8]))

Convert user features to tensors and one hot encode one of user features, and then concatenate them.

In [ ]:
user_feature1_6 = tf.cast(tf.convert_to_tensor(user_dict['user']), tf.float32)
user_feature7 = tf.one_hot(user_dict['user_fav'], len(brand_choice_list))
user = tf.concat([user_feature1_6, user_feature7], 1)

User features are performance, camera, battery, software, RAM, storage, and favorite brand.

In [ ]:
user[0]

<tf.Tensor: shape=(17,), dtype=float32, numpy=
array([3., 5., 3., 1., 3., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
      dtype=float32)>

Create a brand list and os list for one hot encoding.

In [ ]:
phone_dict = {}
brand_list = ['Oppo', 'Asus', 'Infinix', 'Samsung', 'Vivo', 'Huawei', 'Apple', 'Realme', 'Xiaomi', 'Poco']
os_list = ['iOS', 'EMUI', 'Android']
chipset_list = ['unisoc', 'helio','dimensity_6', 'dimensity_7', 'dimensity_8', 'dimensity_9', 'snapdragon_6', 'snapdragon_7', 'snapdragon_8', 'exynos_1', 'exynos_2', 'kirin', 'apple']
price_list = ['_2', '2_3', '3_4', '4_6', '6_8', '8_10', '10_']

Access phone dataset from csv file.<br>
Preprocess the data,  one hot encode some of the features, normalize some of the features.<br>
Then concatenate all the phone data.

In [ ]:
with open('phone_dataset.csv', 'r') as file:
  phone_dict = {'phone_brand': [], 'phone_name': [], 'phone_weight': [], 'phone_os': [], 'phone_chipset': [], 'phone_memory': [], 'phone_ram': [], 'phone_camera': [], 'phone_audio_jack': [], 'phone_battery': [], 'phone_charging': [], 'phone_nfc': [], 'phone_price': []}
  csvFile = csv.reader(file)
  next(csvFile)
  for i,line in enumerate(csvFile):
    phone_dict['phone_brand'].append(brand_list.index(line[1]))
    phone_dict['phone_name'].append(line[2])
    phone_dict['phone_weight'].append(float(line[6]))
    phone_dict['phone_os'].append(os_list.index('iOS' if line[7].startswith('iOS') else 'EMUI' if line[7].startswith('EMUI') else 'Android'))
    phone_dict['phone_chipset'].append(chipset_list.index('unisoc' if 'unisoc' in line[8].lower()
                                                          else 'helio' if 'helio' in line[8].lower()
                                                          else 'dimensity_6' if 'dimensity 6' in line[8].lower()
                                                          else 'dimensity_7' if 'dimensity 7' in line[8].lower()
                                                          else 'dimensity_8' if 'dimensity 8' in line[8].lower()
                                                          else 'dimensity_9' if 'dimensity 9' in line[8].lower()
                                                          else 'snapdragon_6' if 'snapdragon 6' in line[8].lower()
                                                          else 'snapdragon_7' if 'snapdragon 7' in line[8].lower()
                                                          else 'snapdragon_8' if 'snapdragon 8' in line[8].lower()
                                                          else 'exynos_1' if 'exynos 1' in line[8].lower()
                                                          else 'exynos_2' if 'exynos 2' in line[8].lower()
                                                          else 'kirin' if 'kirin' in line[8].lower()
                                                          else 'apple' if 'apple' in line[8].lower() else ''))
    phone_dict['phone_memory'].append(float(line[9]))
    phone_dict['phone_ram'].append(float(line[10]))
    phone_dict['phone_camera'].append([float(line[11]), float(line[12]), float(line[13]), float(line[14])])
    phone_dict['phone_audio_jack'].append(1.0 if line[15] == 'TRUE' else 0.0)
    phone_dict['phone_battery'].append(float(line[16]))
    phone_dict['phone_charging'].append(float(line[17]))
    phone_dict['phone_nfc'].append(1.0 if line[19] == 'TRUE' else 0.0)
    phone_dict['phone_price'].append(price_list.index('_2' if int(line[20]) < 2000000
                                                      else '2_3' if int(line[20]) < 3000000
                                                      else '3_4' if int(line[20]) < 4000000
                                                      else '4_6' if int(line[20]) < 6000000
                                                      else '6_8' if int(line[20]) < 8000000
                                                      else '8_10' if int(line[20]) < 10000000
                                                      else '10_'))

In [ ]:
phone_brand = tf.one_hot(phone_dict['phone_brand'], len(brand_list))
phone_weight = tf.expand_dims(tf.convert_to_tensor(phone_dict['phone_weight']) / max(phone_dict['phone_weight']), 1)
phone_os = tf.one_hot(phone_dict['phone_os'], len(os_list))
phone_chipset = tf.one_hot(phone_dict['phone_chipset'], len(chipset_list))
phone_memory = tf.expand_dims(tf.convert_to_tensor(phone_dict['phone_memory']), 1)  / max(phone_dict['phone_memory'])
phone_ram = tf.expand_dims(tf.convert_to_tensor(phone_dict['phone_ram']), 1) / max(phone_dict['phone_ram'])
phone_camera = tf.convert_to_tensor(phone_dict['phone_camera'])
phone_audio_jack = tf.expand_dims(tf.convert_to_tensor(phone_dict['phone_audio_jack']), 1)
phone_battery = tf.expand_dims(tf.convert_to_tensor(phone_dict['phone_battery']), 1) / max(phone_dict['phone_battery'])
phone_charging = tf.expand_dims(tf.convert_to_tensor(phone_dict['phone_charging']), 1)
phone_nfc = tf.expand_dims(tf.convert_to_tensor(phone_dict['phone_nfc']), 1)
phone_price = tf.one_hot(phone_dict['phone_price'], len(price_list))

In [ ]:
phone = tf.concat([phone_brand, phone_weight, phone_os, phone_chipset, phone_memory, phone_ram, phone_camera, phone_audio_jack, phone_battery, phone_charging, phone_nfc, phone_price], 1)

Phone features are brand, weight, os, chipset, memory, RAM, camera 1, camera 2, camera 3, selfie camera, audio jack, battery, charging, nfc, and price.

In [ ]:
phone[0]

<tf.Tensor: shape=(44,), dtype=float32, numpy=
array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.33643124,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.5       ,  0.5       , 50.        ,
       32.        ,  8.        , 32.        ,  0.        ,  0.575     ,
       80.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ], dtype=float32)>

Create a rating dictionary and a rating list.

In [ ]:
rating_dict = {}

In [ ]:
rating = [[0.0 for _ in range(1507)] for _ in range(96)]

In [ ]:
rating[0][0]

0.0

Access rating dataset from csv file and only save the ratings from the users with information.

In [ ]:
with open('user_ratings.csv', 'r') as file:
  csvFile = csv.reader(file)
  next(csvFile)
  for line in csvFile:
    if (int(line[0])-1) in user_index and line[2]:
      rating[int(line[1])-1][int(line[0])-1] = float(line[2])

In [ ]:
print(rating[17][500])
print(rating[4][1])
print(rating[9][0])

4.0
0.0
0.0


In [ ]:
phone_data = []
user_data = []
rating_data = []

Save the phone data and user data for training only for the rated phone.

In [ ]:
for i,p in enumerate(phone):
  for j,u in zip(user_index, user):
    if rating[i][j] > 0:
      phone_data.append(phone[i])
      user_data.append(user[user_index.index(j)])
      rating_data.append(rating[i][j])

In [ ]:
len(phone_data)

5439

Convert phone data, user data, and rating data to tensors and shuffle them.

In [ ]:
combined_dataset = list(zip(phone_data, user_data, rating_data))
random.shuffle(combined_dataset)
phone_dataset, user_dataset, rating_dataset = zip(*combined_dataset)
phone_dataset, user_dataset, rating_dataset = list(phone_dataset), list(user_dataset), list(rating_dataset)

In [ ]:
phone_dataset = tf.convert_to_tensor(phone_dataset)
user_dataset = tf.convert_to_tensor(user_dataset)
rating_dataset = tf.convert_to_tensor(rating_dataset)

Distribute data into training and validation set.

In [ ]:
train_phone = phone_dataset[int(len(phone_dataset)/4):]
train_user = user_dataset[int(len(user_dataset)/4):]
train_rating = rating_dataset[int(len(rating_dataset)/4):]

In [ ]:
val_phone = phone_dataset[:int(len(phone_dataset)/4)]
val_user = user_dataset[:int(len(user_dataset)/4)]
val_rating = rating_dataset[:int(len(rating_dataset)/4)]

Create a model with two inputs for user data and phone data, the output is the dot product of the two vectors from user_NN and phone_NN which is the predicted rating.

In [ ]:
num_outputs = 32

user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
])

phone_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
])

user_input = tf.keras.layers.Input(shape=(17))
user_vector = user_NN(user_input)
normalized_user = tf.linalg.l2_normalize(user_vector, axis=1)

phone_input = tf.keras.layers.Input(shape=(44))
phone_vector = phone_NN(phone_input)
normalized_phone = tf.linalg.l2_normalize(phone_vector, axis=1)

output = tf.keras.layers.Dot(axes=1)([normalized_user,normalized_phone])
output = tf.keras.layers.Lambda(lambda x: x * 5)(output)

model = tf.keras.models.Model(inputs=[user_input,phone_input], outputs=output)
user_model = tf.keras.models.Model(inputs=user_input, outputs=normalized_user)
phone_model = tf.keras.models.Model(inputs=phone_input, outputs=normalized_phone)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 17)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 44)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   41632     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   48544     ['input_2[0][0]']             
                                                                                              

In [ ]:
user_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 17)]              0         
                                                                 
 sequential (Sequential)     (None, 32)                41632     
                                                                 
 tf.math.l2_normalize (TFOp  (None, 32)                0         
 Lambda)                                                         
                                                                 
Total params: 41632 (162.62 KB)
Trainable params: 41632 (162.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
phone_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 44)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                48544     
                                                                 
 tf.math.l2_normalize_1 (TF  (None, 32)                0         
 OpLambda)                                                       
                                                                 
Total params: 48544 (189.62 KB)
Trainable params: 48544 (189.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mse')

Train the model.

In [ ]:
model.fit([train_user, train_phone], train_rating, epochs=30, validation_data=([val_user, val_phone], val_rating))

Epoch 1/30
128/128 [==============================] - 3s 9ms/step - loss: 0.5049 - val_loss: 0.3358
Epoch 2/30
128/128 [==============================] - 1s 6ms/step - loss: 0.3161 - val_loss: 0.3334
Epoch 3/30
128/128 [==============================] - 1s 5ms/step - loss: 0.3032 - val_loss: 0.3375
Epoch 4/30
128/128 [==============================] - 0s 4ms/step - loss: 0.3049 - val_loss: 0.3324
Epoch 5/30
128/128 [==============================] - 0s 4ms/step - loss: 0.3040 - val_loss: 0.3368
Epoch 6/30
128/128 [==============================] - 0s 3ms/step - loss: 0.2944 - val_loss: 0.3105
Epoch 7/30
128/128 [==============================] - 0s 4ms/step - loss: 0.2984 - val_loss: 0.3160
Epoch 8/30
128/128 [==============================] - 0s 3ms/step - loss: 0.2978 - val_loss: 0.3136
Epoch 9/30
128/128 [==============================] - 0s 4ms/step - loss: 0.2976 - val_loss: 0.3178
Epoch 10/30
128/128 [==============================] - 0s 3ms/step - loss: 0.2914 - val_loss: 0.3115

In [ ]:
model.evaluate([val_user, val_phone], val_rating)

43/43 [==============================] - 0s 2ms/step - loss: 0.3087


0.3087339997291565

Save model in keras format.

In [ ]:
model.save('model.keras')

In [ ]:
user_model.save('user_model.keras')

In [ ]:
phone_model.save('phone_model.keras')

Save model in h5 format.

In [ ]:
model.save('model.h5')

In [ ]:
user_model.save('user_model.h5')

In [ ]:
phone_model.save('phone_model.h5')

Predict rating.

In [ ]:
model.predict([tf.convert_to_tensor([user_data[0]]), tf.convert_to_tensor([phone_data[0]])])

1/1 [==============================] - 0s 226ms/step


array([[4.228704]], dtype=float32)

In [ ]:
rating_data[0]

3.0

In [ ]:
user_data[0]

<tf.Tensor: shape=(17,), dtype=float32, numpy=
array([4., 5., 5., 4., 5., 5., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

Predict rating by dot product of user vector and phone vector.

In [ ]:
user_vector_pred = user_model.predict(tf.expand_dims(user_data[0], 0))
phone_vector_pred = phone_model.predict(tf.expand_dims(phone_data[0], 0))
tf.tensordot(user_vector_pred, phone_vector_pred, axes=(1,1)) * 5

1/1 [==============================] - 0s 67ms/step


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4.228704]], dtype=float32)>

Preprocess data into input compatible with the model.

In [ ]:
def preprocess_user(x):
  one_hot = x[-1]
  one_hot = tf.one_hot(brand_choice_list.index(one_hot), len(brand_choice_list))
  x = tf.cast(tf.convert_to_tensor(x[:-1]), tf.float32)
  return tf.expand_dims(tf.concat([x, one_hot], 0), 0)

In [ ]:
#  Oppo, Asus, Infinix, Samsung, Vivo, Huawei, Apple, Realme, Xiaomi, Poco, lainnya/tidak ada

In [ ]:
a = [5,5,5,5,5,5,'Apple'] # performance, camera, battery, software, RAM, storage, favorite brand
preprocessed_output = preprocess_user(a)

Create a user vector.

In [ ]:
u_vector = user_model.predict(preprocessed_output)

1/1 [==============================] - 0s 22ms/step


In [ ]:
u_vector.shape

(1, 32)

Create phone vectors.

In [ ]:
p_vector = phone_model.predict(phone)

3/3 [==============================] - 0s 5ms/step


In [ ]:
p_vector.shape

(96, 32)

Save phone vectors into csv file.

In [ ]:
with open('phone_vector.csv', 'w') as file:
  csvwriter = csv.writer(file)
  csvwriter.writerows(p_vector)

Predict phone ratings for a user.

In [ ]:
rating_pred_list = []
for p in p_vector:
  rating_pred_list.append(tf.tensordot(u_vector, tf.expand_dims(p, 0), axes=(1,1)) * 5)

Get phone name.

In [ ]:
phone_pred_list = phone_dict['phone_name']

Sort ratings to get top 10 recommended phones for a user.

In [ ]:
top_10_phone = list(zip(rating_pred_list, phone_pred_list))
top_10_phone.sort(reverse=True)
rating_pred_list, phone_pred_list = zip(*top_10_phone)
rating_pred_list, phone_pred_list = list(rating_pred_list), list(phone_pred_list)

Print out top 10 recommended phones.

In [ ]:
top_10_list = []
for i in range(10):
  top_10_list.append(phone_pred_list[i])
  print(i+1, phone_pred_list[i], rating_pred_list[i].numpy())

1 iPhone 15 Plus [[4.4538746]]
2 iPhone 15 [[4.453345]]
3 iPhone 13 [[4.451524]]
4 iPhone 14 Plus [[4.4514613]]
5 Infinix GT 20 Pro [[4.447506]]
6 Samsung Galaxy S24 Ultra [[4.438769]]
7 Samsung Galaxy A35 [[4.433384]]
8 Samsung Galaxy S24 [[4.4322104]]
9 iPhone 15 Pro Max [[4.4306607]]
10 iPhone 14 Pro Max [[4.4306607]]


In [ ]:
top_10_list

['iPhone 15 Plus',
 'iPhone 15',
 'iPhone 13',
 'iPhone 14 Plus',
 'Infinix GT 20 Pro',
 'Samsung Galaxy S24 Ultra',
 'Samsung Galaxy A35',
 'Samsung Galaxy S24',
 'iPhone 15 Pro Max',
 'iPhone 14 Pro Max']

Calculate square distance between phone vectors to get related phones.

In [ ]:
top_distance = []
top_distance_name = []
for i,x in enumerate(p_vector):
  all_distance = []
  phone_pred_list = phone_dict['phone_name']
  for j,y in enumerate(p_vector):
    if tf.math.reduce_all(tf.math.equal(x,y)):
      continue
    else:
      all_distance.append(tf.math.reduce_sum(tf.math.square(x-y)))
  phone_pred_list = phone_pred_list[:i] + phone_pred_list[i+1:]
  combined_dist = list(zip(all_distance, phone_pred_list))
  combined_dist.sort()
  all_distance, phone_pred_list = zip(*combined_dist)
  all_distance, phone_pred_list = list(all_distance), list(phone_pred_list)
  top_distance.append(all_distance[:10])
  top_distance_name.append(phone_pred_list[:10])

In [ ]:
top_distance[0]

[<tf.Tensor: shape=(), dtype=float32, numpy=2.7269867e-05>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00020949422>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00036774497>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0004645838>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0005682481>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00060009747>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00067274587>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0008121567>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0008850064>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0009351555>]

In [ ]:
top_distance_name[0]

['Huawei Pura 70',
 'Vivo Y100',
 'Oppo Reno 11F',
 'Vivo V30e',
 'Oppo A60',
 'Asus Zenfone 10',
 'Vivo V29e',
 'Vivo V30',
 'Realme 12+ 5G',
 'Oppo Reno 10 Pro']

Save related phones data into csv file.

In [ ]:
with open('related_phone.csv', 'w') as file:
  csvwriter = csv.writer(file)
  csvwriter.writerow(['phone_name', 'related1', 'related2', 'related3', 'related4', 'related5', 'related6', 'related7', 'related8', 'related9', 'related10'])
  for i in range(96):
    csvwriter.writerow([phone_dict['phone_name'][i], top_distance_name[i][0], top_distance_name[i][1], top_distance_name[i][2], top_distance_name[i][3], top_distance_name[i][4], top_distance_name[i][5], top_distance_name[i][6], top_distance_name[i][7], top_distance_name[i][8], top_distance_name[i][9]])